# CIRED 2024 Article

This notebook contains the simulation presented in the article submitted to the CIRED 2024 conference by Roseau Technologies.

You can find the article in pdf format here: [Roseau_Load_Flow_CIRED_2024.pdf](./Article/Roseau_Load_Flow_CIRED_2024.pdf)

<!-- TODO: Add the published pdf when available and add citation -->

The goal of the article is study the hosting capacity of a low-voltage grid with a non-firm connection of PV farm.
The flexibility of the non-firm connection is modelled with the
[flexible loads](https://roseau-load-flow.roseautechnologies.com/models/Load/FlexibleLoad/index.html)
built into the _Roseau Load Flow_ solver.

The low-voltage grid chosen to perform this study is the following:
https://roseau-load-flow.roseautechnologies.com/_static/Network/LVFeeder00939.html


In [ ]:
from IPython.display import IFrame

url = "https://roseau-load-flow.roseautechnologies.com/_static/Network/LVFeeder00939.html"
IFrame(url, width=1000, height=500)

In [ ]:
import cmath
from pathlib import Path

import numpy as np
import pandas as pd
import roseau.load_flow as lf
import seaborn as sns

sns.set_theme()
DATA_DIR = Path("data")
RESULTS_DIR = Path("results")

Activate the public [license](https://roseau-load-flow.roseautechnologies.com/License.html) of _Roseau Load Flow_.


In [ ]:
%load_ext dotenv
%dotenv

## Read the loads profiles

The load profiles are randomly generated using [pyLPG](https://github.com/FZJ-IEK3-VSA/pylpg)[1]

```bibliography
[1] Pflugradt et al., (2022). “LoadProfileGenerator: An Agent-Based Behavior Simulation
for Generating Residential Load Profiles”. Journal of Open Source Software, 7(71), 3574,
https://doi.org/10.21105/joss.03574.
```

Here we have load profiles of the full year 2020 with a 1-minute time step generated in advance
and stored in an HDF file. We load the profiles into a dataframe and resample it into a 10-minute
time series.


In [ ]:
load_profiles_original: pd.DataFrame = pd.read_hdf(DATA_DIR / "10_random_load_curves.hdf")
# Resample to 10 minutes
load_profiles = load_profiles_original.resample("10min").mean()
load_profiles.describe()

We can take a look at how these load profiles roughly look like by plotting the first day:


In [ ]:
first_day_mask: "pd.Series[bool]" = load_profiles.index.date == pd.to_datetime("2020-01-01").date()
first_day = load_profiles.loc[first_day_mask]
first_day_plot_df = first_day.melt(var_name="House", value_name="Power (kW)", ignore_index=False).reset_index(
    names="Time"
)
g = sns.relplot(
    first_day_plot_df,
    x="Time",
    y="Power (kW)",
    style="House",
    hue="House",
    height=5,
    aspect=2.5,
    kind="scatter",
    facet_kws={"sharey": False},
)
g.figure.autofmt_xdate()
g

In [ ]:
load_profile_names = load_profiles.columns.tolist()
load_profile_names

## Read the generation profile

The PV power profile are generated using the Photovoltaic Geographical Information System (PVGIS)[2]

```bibliography
[2] Thomas Huld, Richard Müller, Attilio Gambardella, "A new solar radiation database for
estimating PV performance in Europe and Africa," Solar Energy, Volume 86, Issue 6, 2012,
Pages 1803-1815, ISSN 0038-092X, https://doi.org/10.1016/j.solener.2012.03.006.
```

Here we have a single PV profile of the full year 2019 with a 5-minute sample time stored in a
CSV file. We load the profile into a dataframe and resample it into a 10-minute time series.

The data is stored in the form of a coefficient between 0 and 1 that represents the generated
power as a percentage of the rated power of the inverter.


In [ ]:
# This data is for the year 2019 but the loads profiles are for the year 2020 which is a leap year
# This data is one day short of a full year 2020 (equivalent to 144 records of 10 minutes)
producer_profile_original = pd.read_csv(DATA_DIR / "PV_Profile_PVGIS_1.csv", index_col=0, parse_dates=["HORODATE"])
# The production data is sampled every 5 minutes, we need to resample it to 10 minutes
# Note: using resample() method produces NaN values around the DST transition so we resample with iloc
producer_profile = producer_profile_original.iloc[0::2]["COEFFICIENT_AJUSTE"].rename("coefficient")
# Replace the index of the producer_profile series by the index of the load_profiles dataframe
producer_profile.index = load_profiles.index[: len(producer_profile)]
producer_profile.describe()

## Create the network with a flexible PV generator

The farthest bus from the substation is `LVBus004115` (see the map above).
We choose this bus to connect our PV installation.


In [ ]:
def create_network(s_max: int, projection: str) -> lf.ElectricalNetwork:
    """Create a network with a non-firm PV generator represented by the load with ID "producer".

    Args:
        s_max: The rated power of the inverter of the non-firm PV installation.
        projection: The type of the projection used ("euclidean", "keep_p", or "keep_q").

    Returns:
        The created network.
    """
    en = lf.ElectricalNetwork.from_catalogue("LVFeeder00939", "summer")

    u_max = 1.085 * 230  # The voltage limit is set to the nominal voltage + 8.5%
    uq_up, uq_max = u_max - 10, u_max - 5  # 10V before the limit, the Q(U) control activates
    up_up, up_max = u_max - 5, u_max  # 5V before the limit, the P(U) control activates

    # A PV inverter is modelled with a constant-power load with negative active power
    lf.PowerLoad(
        "producer",
        bus=en.buses["LVBus004115"],
        phases="abcn",  # three-phase PV inverter
        powers=[-s_max, -s_max, -s_max],
        flexible_params=[
            lf.FlexibleParameter.pq_u_production(
                up_up=up_up,
                up_max=up_max,
                uq_min=207,
                uq_down=212,
                uq_up=uq_up,
                uq_max=uq_max,
                s_max=s_max,
                type_proj=projection,
            )
            for _ in range(3)  # Three flexible parameters, one per phase
        ],
    )
    vs = en.sources["VoltageSource"]
    vs.voltages *= 1.05  # Set the voltage of the upstream MV grid
    return en

## Run the simulation

We run three simulations, one for each projection type.

1. Set the `projection` variable in the following cell to `"euclidean"`, `"keep_p"`, or `"keep_q"` then
   execute all the cells below to run the time series simulation and save its results to the disk.
2. Repeat this procedure for the other two projections.
3. At the end, we will have a "results" folder with the time series results as well as file
   "statistics.csv" for each projection.


In [ ]:
lf_results = []
bus_results = []
line_results = []
transformer_results = []
loads_results = []

# Choose the projection type (uncomment the corresponding line)
# -------------------------------------------------------------
projection = "euclidean"
# projection = "keep_p"
# projection = "keep_q"

# We consider that the PV inverters operate with a cos Phi = 0.98 by default
complex_power = cmath.exp(1j * cmath.acos(-0.98))
producer_s_max = 24_000

# Create the network and add the non-firm producer
# ------------------------------------------------
en = create_network(producer_s_max, projection)
non_firm_producer: lf.PowerLoad = en.loads["producer"]

# We also add a normal single-phase producer on bus LVBus004116 to help create voltage constraints
firm_producer: lf.PowerLoad = en.loads["LVBus004116_production"]


# Run the time series simulation
# ------------------------------
load: lf.PowerLoad
indices = range(min(load_profiles.shape[0], producer_profile.shape[0]))
for idx in indices:
    # Get the next time step
    ts = load_profiles.index[idx]
    ts_prod = producer_profile.index[idx]

    # Update the power of all loads
    col_idx = 0
    prod_power_coef = max(producer_profile.at[ts_prod], 1e-3)
    for load in en.loads.values():
        if load.id.endswith("_consumption"):
            # Load powers are read from the load_profiles data frame
            load.powers = [load_profiles.at[ts, load_profile_names[col_idx]] * 1e3]
            col_idx += 1
        elif load.id == firm_producer.id:
            # The normal (firm) producer's power is 12kVA multiplied by the PV profile coefficient
            load.powers = [12_000 * complex_power * prod_power_coef]
        elif load.id == non_firm_producer.id:
            # The non-firm producer's power is 24kVA (per phase) multiplied by the PV profile coefficient
            load.powers = [producer_s_max * complex_power * prod_power_coef] * 3  # three-phase
        else:
            # Other existing dummy producers are not active
            load.powers = [0]  # Set powers of other producers to 0

    # Run the load flow with the updated loads
    iterations, residual = en.solve_load_flow()

    # Store the results of the load flow
    lf_results.append({"timestamp": ts, "iterations": iterations, "residual": residual})

    # Store the results of all LV buses
    for bus in en.buses.values():
        if bus.id == "MVLV03045":
            continue  # Skip the substation
        res_voltages = abs(bus.res_voltages.m)
        res_violated = (res_voltages < bus.min_voltage.m) | (res_voltages > bus.max_voltage.m)
        vuf = bus.res_voltage_unbalance().m
        for ph, voltage, violated in zip(bus.voltage_phases, res_voltages, res_violated, strict=True):
            bus_results.append({
                "timestamp": ts,
                "bus": bus.id,
                "phase": ph,
                "type": "voltage",
                "value": voltage,
                "violated": violated,
                "unbalance_factor": vuf,
            })

    # Store the results of the lines and transformers
    for branch in en.branches.values():
        if isinstance(branch, lf.Line):
            res_currents = abs(branch.res_series_currents.m)
            res_violated = res_currents > branch.parameters.max_current.m
            for ph, current, power, violated in zip(
                branch.phases, res_currents, branch.res_power_losses.m.real, res_violated, strict=True
            ):
                line_results.append({
                    "timestamp": ts,
                    "branch": branch.id,
                    "phase": ph,
                    "type": "current",
                    "value": current,
                    "violated": violated,
                })
                line_results.append({
                    "timestamp": ts,
                    "branch": branch.id,
                    "phase": ph,
                    "type": "power",
                    "value": power,
                    "violated": violated,
                })
        elif isinstance(branch, lf.Transformer):
            res_powers1, res_powers2 = (p.m.sum() for p in branch.res_powers)
            max_power = branch.parameters.max_power.m
            violated = abs(res_powers1) > max_power or abs(res_powers2) > max_power
            ph1, ph2 = branch.phases1, branch.phases2
            transformer_results.append({
                "timestamp": ts,
                "branch": branch.id,
                "phase": ph1,
                "type": "active_power",
                "value": res_powers1.real,
                "violated": violated,
                "side": "primary",
            })
            transformer_results.append({
                "timestamp": ts,
                "branch": branch.id,
                "phase": ph1,
                "type": "reactive_power",
                "value": res_powers1.imag,
                "violated": violated,
                "side": "primary",
            })
            transformer_results.append({
                "timestamp": ts,
                "branch": branch.id,
                "phase": ph2,
                "type": "active_power",
                "value": res_powers2.real,
                "violated": violated,
                "side": "secondary",
            })
            transformer_results.append({
                "timestamp": ts,
                "branch": branch.id,
                "phase": ph2,
                "type": "reactive_power",
                "value": res_powers2.imag,
                "violated": violated,
                "side": "secondary",
            })

    # Store the results of the non-firm PV
    load_theoretical_power = non_firm_producer.powers.m.sum()
    res_flexible_power = non_firm_producer.res_flexible_powers.m.sum()
    loads_results.append({
        "timestamp": ts,
        "load": non_firm_producer.id,
        "phase": non_firm_producer.phases,
        "type": "active_power",
        "value": res_flexible_power.real,
        "flexible": True,
    })
    loads_results.append({
        "timestamp": ts,
        "load": non_firm_producer.id,
        "phase": non_firm_producer.phases,
        "type": "reactive_power",
        "value": res_flexible_power.imag,
        "flexible": True,
    })
    loads_results.append({
        "timestamp": ts,
        "load": non_firm_producer.id,
        "phase": non_firm_producer.phases,
        "type": "active_power",
        "value": load_theoretical_power.real,
        "flexible": False,
    })
    loads_results.append({
        "timestamp": ts,
        "load": non_firm_producer.id,
        "phase": non_firm_producer.phases,
        "type": "reactive_power",
        "value": load_theoretical_power.imag,
        "flexible": False,
    })

## Simulation results and statistics

We read the results into data frames and calculate some statistics including the annual energy curtailment


In [ ]:
print("Reading load flow results ...")
lf_results_df = pd.DataFrame(lf_results)
print("Reading bus results ...")
bus_results_df = pd.DataFrame(bus_results)
print("Reading line results ...")
line_results_df = pd.DataFrame(line_results)
print("Reading transformer results ...")
transformer_results_df = pd.DataFrame(transformer_results)
print("Reading loads results ...")
load_results_df = pd.DataFrame(loads_results)
print("Done")

In [ ]:
def calculate_energy(df: pd.DataFrame) -> float:
    """E(Wh) = ∑ P(W) * 10 (min) / 60 (hour)."""
    powers = -df.loc[(df["type"] == "active_power") & (df["load"] == "producer"), "value"]
    total_wh_energy = powers.sum() / 6

    # # Alternatively, we can integrate using the trapezoid method
    # from scipy import integrate
    # total_wh_energy = integrate.trapezoid(powers.values, dx=1 / 6)

    return total_wh_energy

In [ ]:
# Statistics
statistics = []

# Producer
theoretical_energy = calculate_energy(load_results_df.loc[~load_results_df["flexible"]])
flexible_energy = calculate_energy(load_results_df.loc[load_results_df["flexible"]])
curtailment = (theoretical_energy - flexible_energy) / theoretical_energy
statistics.append({"name": "Theoretical producer energy", "value": theoretical_energy, "unit": "Wh"})
statistics.append({"name": "Flexible producer energy", "value": flexible_energy, "unit": "Wh"})
statistics.append({"name": "Producer energy curtailment", "value": curtailment * 100, "unit": "%"})

# Buses
statistics.append({"name": "Voltage violations count", "value": bus_results_df["violated"].sum() // 3, "unit": ""})
statistics.append({"name": "Maximum voltage", "value": bus_results_df["value"].max(), "unit": "V"})
statistics.append({"name": "Minimum voltage", "value": bus_results_df["value"].min(), "unit": "V"})
statistics.append({"name": "Average voltage", "value": bus_results_df["value"].mean(), "unit": "V"})
statistics.append({"name": "Maximum VUF", "value": bus_results_df["unbalance_factor"].max(), "unit": "%"})
statistics.append({"name": "Minimum VUF", "value": bus_results_df["unbalance_factor"].min(), "unit": "%"})
statistics.append({"name": "Average VUF", "value": bus_results_df["unbalance_factor"].mean(), "unit": "%"})

# Lines
statistics.append({
    "name": "Maximum line current",
    "value": line_results_df.loc[
        (line_results_df["phase"].isin(tuple("abc"))) & (line_results_df["type"] == "current"), "value"
    ].max(),
    "unit": "A",
})
statistics.append({
    "name": "Maximum neutral current",
    "value": line_results_df.loc[
        (line_results_df["phase"] == "n") & (line_results_df["type"] == "current"), "value"
    ].max(),
    "unit": "A",
})

# Transformer
tr_p = transformer_results_df.loc[transformer_results_df["type"] == "active_power"]
tr_q = transformer_results_df.loc[transformer_results_df["type"] == "reactive_power"]
statistics.append({"name": "Maximum transformer active power", "value": np.max(abs(tr_p["value"])), "unit": "W"})
statistics.append({"name": "Maximum transformer reactive power", "value": (np.max(abs(tr_q["value"]))), "unit": "VAr"})
statistics.append({
    "name": "Transformer injected energy",
    "value": tr_p.loc[tr_p["side"] == "primary", "value"].sum() / 6,
    "unit": "Wh",
})

# Energy losses
statistics.append({
    "name": "Lines losses",
    "value": (line_losses := line_results_df.loc[line_results_df["type"] == "power", "value"].sum() / 6),
    "unit": "Wh",
})
statistics.append({
    "name": "Transformer losses",
    "value": (transformer_losses := tr_p["value"].sum() / 6),
    "unit": "Wh",
})
statistics.append({"name": "Total losses", "value": line_losses + transformer_losses, "unit": "Wh"})

statistics_df = pd.DataFrame(statistics)
statistics_df

## Save the results

We then save the time series and statistics results to the disk for later analysis


In [ ]:
(RESULTS_DIR / projection).mkdir(parents=True, exist_ok=True)
print("Saving the network ...")
en.to_json(RESULTS_DIR / projection / "network.json", include_results=False)
print("Saving load flow results ...")
lf_results_df.to_csv(RESULTS_DIR / projection / "load_flow.csv", index=False)
print("Saving bus results ...")
bus_results_df.to_csv(RESULTS_DIR / projection / "bus.csv", index=False)
print("Saving line results ...")
line_results_df.to_csv(RESULTS_DIR / projection / "line.csv", index=False)
print("Saving transformer results ...")
transformer_results_df.to_csv(RESULTS_DIR / projection / "transformer.csv", index=False)
print("Saving load results ...")
load_results_df.to_csv(RESULTS_DIR / projection / "load.csv", index=False)
print("Saving the statistics ...")
statistics_df.to_csv(RESULTS_DIR / projection / "statistics.csv", index=False)
print("Done")